In [5]:
import logging
import json
import requests
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Polygon, Point
from folium import plugins
import folium
import numpy
import matplotlib.pyplot as plt
import utm
import random 
from typing import Tuple
import time

OPTIMIZED_ROUTE_URL = "http://localhost:8002/optimized_route" 
DEFAULT_HEADERS = {"Content-type": "application/json"}

# Use six degrees of precision when using Valhalla for routing
VALHALLA_PRECISION = 1.0 / 1e6

EARTH_RADIUS = 6371. * 1000. # In meters
center = [55.39594, 10.38831]

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [30]:
class DataPoint:
    
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude
        
class WayPoint(DataPoint):
    
    def __init__(self, latitude: float, longitude: float, duration: float = 0., variance: float = 0.):
        super().__init__(latitude = latitude, longitude = longitude)
        self.variance = variance
        self.duration = duration
        
class ValhallaInterface:

    def __init__(self) -> None:
        pass


    def decode_polyline(self, polyline_string):
        index = 0; latitude = 0; longitude = 0
        coordinates = []
        changes = {"latitude": 0, "longitude": 0}
        # Coordinates have variable length when encoded, so just keep
        # track of whether we have hit the end of the string. In each
        # while loop iteration a single coordinate is decoded.
        while index < len(polyline_string):
            # Gather latitude/longitufe changes, store them in a dictionary to apply them later
            for unit in ["latitude", "longitude"]: 
                shift, result = 0, 0
                while True:
                    byte = ord(polyline_string[index]) - 63
                    index += 1
                    result |= (byte & 0x1f) << shift
                    shift += 5
                    if not byte >= 0x20:
                        break
                if (result & 1):
                    changes[unit] = ~(result >> 1)
                else:
                    changes[unit] = (result >> 1)
            latitude += changes["latitude"]
            longitude += changes["longitude"]
            coordinates.append(
                [VALHALLA_PRECISION * latitude, VALHALLA_PRECISION * longitude],
            )
        return coordinates

    def send_optimized_route_request(self, dp1, dp2):    
        def build_optimized_route_request(dp1, dp2):
            return json.dumps({
                "locations":[
                    # Start location
                    {"lat": dp1.latitude, "lon": dp1.longitude},
                    # End location
                    {"lat": dp2.latitude, "lon": dp2.longitude},
                ],
                "costing": "pedestrian",
                "directions_options": {
                    "units":"kilometers"
                },
            })
        d = build_optimized_route_request(dp1, dp2)
        response = requests.post(
            OPTIMIZED_ROUTE_URL,
            data = d,
            headers = DEFAULT_HEADERS,
        )
        if response.status_code == 200:
            content = json.loads(response.content)
        else:
            content = None
        return content

def setup_map(center, zoom_start = 14, tiles: str = "cartodbdark_matter"):
    map_ = folium.Map(
        location = center,
        zoom_start = zoom_start,
#         tiles = tiles,
    )
    plugins.Fullscreen(
        position = "topleft"
    ).add_to(map_)
    plugins.Draw(
        filename="placeholder.geojson",
        export = True,
        position = "topleft"
    ).add_to(map_)
    return map_

def plot_point(datapoint, center, color, radius = 5.0, opacity = 1, map_ = None):
    folium.CircleMarker(
        [datapoint.latitude, datapoint.longitude],
        radius = radius,
        color = color,
        opacity = opacity,
        popup = f"...",
    ).add_to(map_)
    return map_
    
def plot_polyline(datapoints, center, color, weight: float = 2.0, opacity: float = 1, map_ = None):
        lst = []
        for datapoint in datapoints:
            lst.append([
                datapoint.latitude,
                datapoint.longitude,
            ])
        folium.PolyLine(
            lst,
            color = color,
            weight = weight,
            opacity = opacity,
            popup = f"...",
        ).add_to(map_)
        return map_

    
def test_valhalla(start_dp, end_dp):
    d = json.dumps({
        "locations":[
            {"lat": start_dp.latitude, "lon": start_dp.longitude},
            {"lat": end_dp.latitude, "lon": end_dp.longitude},
#             {"lat": 55.39594, "lon": 10.38831},
#             {"lat": 55.39500, "lon": 10.38800},
        ],
        "costing": "pedestrian",
        "directions_options": {
            "units":"kilometers"
        },
    })
    response = requests.post(
        OPTIMIZED_ROUTE_URL,
        data = d,
        headers = DEFAULT_HEADERS,
    )
    return response

def haversine_distance(
        lat_1: float, lon_1: float,
        lat_2: float, lon_2: float,
    ) -> float:
    """Calculate the 'Haversine' (great-circle) distance in meters between two locations
    / (latitude, longitude) points.

    Args:
        lat_1 (float): Latitude of location 1.
        lon_1 (float): Longitude of location 1.
        lat_2 (float): Latitude of location 2.
        lon_2 (float): Longitude of location 2.

    Returns:
        float: The distance between location 1 and 2 in meters. 
    """
    d_lat = np.radians(lat_1 - lat_2)
    d_lon = np.radians(lon_1 - lon_2)
    lat1 = np.radians(lat_1)
    lat2 = np.radians(lat_2)
    a = np.sin(d_lat / 2) * np.sin(d_lat / 2) + \
        np.sin(d_lon / 2) * np.sin(d_lon / 2) * np.cos(lat1) * np.cos(lat2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = EARTH_RADIUS * c
    return d

def rand_24_bit() -> int:
    """Returns a random 24-bit integer"""
    return random.randrange(0, 16**6)


def color_dec() -> int:
    """Alias of rand_24 bit()"""
    return rand_24_bit()


def color_hex(num: int = rand_24_bit()) -> str:
    """Returns a 24-bit int in hex"""
    return "%06x" % num


def color_rgb(num: int = rand_24_bit()) -> Tuple[int, int, int]:
    """Returns three 8-bit numbers, one for each channel in RGB"""
    hx = color_hex(num)
    barr = bytearray.fromhex(hx)
    return (barr[0], barr[1], barr[2])

class Path:
    
    def __init__(self, waypoints, time_delta, time_delta_variance, avg_speed, var_speed, datapoint_variance):
        self.waypoints = waypoints
        self.time_delta = time_delta
        self.time_delta_variance = time_delta_variance
        self.avg_speed = avg_speed
        self.var_speed = var_speed
        self.datapoint_variance = datapoint_variance
        self.vi = ValhallaInterface()
        self.path, self.ts = self.create_path()
    
    def stop_point(self, x, y, duration, _):
        # dts = [0]; _dts = [0]; dxs = [0]; counter = 0
        dts = []; _dts = 0.; dxs = []; counter = 0
        lats = []; lons = []
        while True:
            dt = self.time_delta + np.abs(np.random.normal(0, self.time_delta_variance))
            dt_next = _dts + dt 
            if dt_next > duration:        
                break
            else:
                _dts += dt        
                dts.append(dt)        
                lats.append(x +  np.random.normal(0, self.datapoint_variance))
                lons.append(y + np.random.normal(0, self.datapoint_variance))
                counter += 1        
        return list(zip(lats, lons)), dts
    
    def create_segment(self, waypoint0, waypoint1):
        content = self.vi.send_optimized_route_request(
            dp1 = waypoint0,
            dp2 = waypoint1,
        )
        polyline = self.vi.decode_polyline(
            content["trip"]["legs"][0]["shape"],
        )
        utm_xs = []; utm_ys = []
        for coord in polyline[1:-1]:    
            x, y, _, _ = utm.from_latlon(coord[0], coord[1])
            utm_xs.append(x); utm_ys.append(y)
        segment = list(zip(utm_xs, utm_ys))
        linestring, dts = self.interpolate(
            LineString(segment),
        )
        return list(linestring.coords), dts
      
    
    def create_path(self):
        self.segments = []; self.timestamps = []
        if len(self.waypoints) >= 2:
            waypoint0 = self.waypoints[0]; waypoint1 = self.waypoints[1]
            for i in range(1, len(self.waypoints)):
                # Convert lat/lon to utm coordinates
                if i == 1:
                    start_x, start_y, _, _ = utm.from_latlon(
                        waypoint0.latitude, waypoint0.longitude,
                    )
                    sp_start, dts_start = self.stop_point(
                        start_x, start_y,
                        waypoint0.duration, waypoint0.variance,
                    )
#                     print("start segment length: ", len(sp_start))
#                     print("start dts length    : ", len(dts_start))

                end_x, end_y, _, _ = utm.from_latlon(
                    waypoint1.latitude, waypoint1.longitude,
                )
                sp_end, dts_end = self.stop_point(
                    end_x, end_y,
                    waypoint1.duration, waypoint1.variance,
                )
#                 print("end segment length: ", len(sp_end))
#                 print("end dts length    : ", len(dts_end))
                middle_segment, segment_dts = self.create_segment(
                    waypoint0 = self.waypoints[i - 1],
                    waypoint1 = self.waypoints[i],
                )

                if i == 1:
                    anchor_start = [[start_x, start_y]]
                    anchor_end = [[end_x, end_y]]
                    segment = \
                        sp_start + anchor_start + \
                        middle_segment + \
                        [[end_x, end_y]] + sp_end
#                     anchor_start_dts = self.get_anchor_dts(sp_start, anchor_start, middle_segment[0])
#                     anchor_end_dts = self.get_anchor_dts(sp_end, anchor_end, middle_segment[-1])
#                     print("dts start: ", dts_start)
                    segment_dts[0] = \
                        self.time_delta + np.abs(np.random.normal(0, self.time_delta_variance))
#                     self.avg_speed / np.linalg.norm(
#                         np.array(anchor_start) - np.array(sp_start[-1]),
#                         axis = 1
#                     )[0]
#                     print([0])
#                     print(dts_start)
#                     print(segment_dts)
#                     print(dts_end)
                    dts_start[0] = 0
                    dt0 = self.time_delta + np.abs(np.random.normal(0, self.time_delta_variance))
                    dt1 = self.time_delta + np.abs(np.random.normal(0, self.time_delta_variance))
                    # add transitions: start --add dt--> segment --add dt--> end
                    dts = dts_start + [dt0] + segment_dts + [dt1] + dts_end
#                     dts = [0] + dts_start + segment_dts + dts_end
                else:
                    segment = \
                        middle_segment + \
                        [[end_x, end_y]] + sp_end
                    segment_dts[0] = \
                        self.time_delta + np.abs(np.random.normal(0, self.time_delta_variance))
                    dt0 = self.time_delta + np.abs(np.random.normal(0, self.time_delta_variance))
                    # add transitions: segment --add dt--> end
                    dts = segment_dts + [dt0] + dts_end
                # print("middle segment length: ", len(segment))
                # print("middle dts length    : ", len(dts))
                if len(self.waypoints) > (i + 1):
                    latitude, longitude = utm.to_latlon(sp_end[-1][0], sp_end[-1][1], 32, "U")
                    waypoint0 = WayPoint(
                        latitude = latitude,
                        longitude = longitude,
                        variance = self.waypoints[i].variance,
                        duration = self.waypoints[i].duration,
                    )
                    # Use the next waypoint
                    waypoint1 = self.waypoints[i + 1]
                self.segments.append(segment)
                self.timestamps.append(dts)
        else:
            raise ValueError
        _path = []; _timestamps = []
        for ts in self.timestamps:
            _timestamps.extend(ts)
        for segment in self.segments:
            _path.extend(segment)
        return LineString(_path), _timestamps

    def interpolate(
        self,
        linestring: LineString,
    ) -> LineString:
        # original route linestring with values: [1:-1]
        dxs, dts = self.segment_subdivision(
            segment = linestring,
            time_delta = self.time_delta,
            time_delta_variance = self.time_delta_variance,
            avg_speed = self.avg_speed,
            var_speed = self.var_speed,
        )
        points = [linestring.interpolate(dxs[i], normalized = True) for i in range(len(dxs))]
        points_ = []
        for point in points:
            rand = np.random.normal(0, self.datapoint_variance, size = 2)
            points_.append(
                [point.x + rand[0], point.y + rand[1]]
            )
        return LineString(points_), dts

    def segment_subdivision(self, segment, time_delta, time_delta_variance, avg_speed, var_speed):
        dxs = [0]; dts = [0]; counter = 0
        ls = LineString(segment); ls_length = ls.length
        while True:
            dt = time_delta + np.abs(np.random.normal(0, time_delta_variance))
            dx = (avg_speed + np.random.normal(0, var_speed)) * dt
            dx_next = dxs[counter] + dx 
            if dx_next > ls_length:        
                break
            else:
                dts.append(dt)        
                dxs.append(dx_next)
                counter += 1
        normalized_dxs = [val / ls_length for val in dxs]
        return normalized_dxs, dts
    
    def to_latlon(self):
        lats = []
        lons = []
        if self.path is not None:
            for coord in self.path.coords:
                lat, lon = utm.to_latlon(coord[0], coord[1], 32, "U")
                lats.append(lat)
                lons.append(lon)
        return list(zip(lats, lons))

In [45]:
start_dp = DataPoint(
    latitude = 55.363451206145925,
    longitude = 10.412869042981713,
)
middle_dp = DataPoint(
    latitude = 55.373384555955425,
    longitude = 10.40937928427195,
)

end_dp = DataPoint(
    latitude = 55.37427658758254,
    longitude = 10.403394466623759,
)

waypoints = [
    WayPoint(
        latitude = 55.363451206145925,
        longitude = 10.412869042981713,
        duration = 120, # In seconds
        variance = 14, # In meters
    ),
    WayPoint(
        latitude = 55.373384555955425,
        longitude = 10.40937928427195,
        duration = 120, # Seconds
        variance = 14, # In meters
    ),
    WayPoint(
        latitude = 55.37427658758254,
        longitude = 10.403394466623759,
        duration = 120, # Seconds
        variance = 14, # In meters
    )
]

avg_speed = 5.                      # Average human walking speed in km/h
avg_speed = 5. * (1000 / (60 * 60)) # Average human walking speed in m/s
var_speed = 0.5
var_speed = 0.5 * (1000 / (60 * 60)) # Walking speed in m/s

time_delta = 10.
time_delta_variance = 25.

# samples = 2     # Count 
# timeperiod = 10 # In seconds between datapoints
# sampling_rate = samples / timeperiod  # Desired number of samples per second
# print("Sampling rate: ", sampling_rate)
# distance = avg_speed * timeperiod
# distance

# path = Path(
#     waypoints = waypoints,
#     time_delta = 1.,
#     time_delta_variance = 1.,
#     avg_speed = avg_speed,
#     var_speed = var_speed,
#     datapoint_variance = 15,
# )


map_ = setup_map(center)
map_ = plot_point(start_dp, center, color = "red", map_ = map_)
map_ = plot_point(middle_dp, center, color = "green", map_ = map_)
map_ = plot_point(end_dp, center, color = "blue", map_ = map_)

for _ in range(10):
    random.seed(time.time())
    color = "#" + str(color_hex(num = rand_24_bit()))
    path = Path(
        waypoints = waypoints,
        time_delta = time_delta,
        time_delta_variance = time_delta_variance,
        avg_speed = avg_speed,
        var_speed = var_speed,
        datapoint_variance = 20,
    )
    map_ = plot_polyline(
        [DataPoint(x, y) for x, y in list(path.to_latlon())],
        center,
        color = color,
        map_ = map_,
    )
map_
# print("TS length  : ", len(path.ts))
# print("Path length: ", len(path.path.coords))
# plt.scatter(np.cumsum(path.ts), [i for i, j in list(path.path.coords)])
# plt.scatter(np.cumsum(path.ts), [i for i, j in list(path.path.coords)])
# print()
# for item in path.ts:
#     print(item)


In [ ]:
polyline = [
    [start_dp.latitude, start_dp.longitude],
    [middle_dp.latitude, middle_dp.longitude],
    [end_dp.latitude, end_dp.longitude],
]
utm_xs = []; utm_ys = []
for coord in polyline:    
    x, y, _, _ = utm.from_latlon(coord[0], coord[1])
    utm_xs.append(x); utm_ys.append(y)
segment = list(zip(utm_xs, utm_ys))

def random_interpolation(time_delta, time_delta_variance, avg_speed, var_speed):
    dxs = [0]; dts = [0]; counter = 0
    ls = LineString(segment); ls_length = ls.length
    while True:
        dt = time_delta + np.abs(np.random.normal(0, time_delta_variance))
        dx = avg_speed + np.random.normal(0, var_speed) * dt
        dx_next = dxs[counter] + dx 
        if dx_next > ls_length:        
            break
        else:
            dts.append(dts[counter] + dt)        
            dxs.append(dx_next)
            counter += 1
    normalized_dxs = [val / ls_length for val in dxs] 
    return normalized_dxs, dts

dxs, dts = random_interpolation(
    time_delta = time_delta,
    time_delta_variance = time_delta_variance,
    avg_speed = avg_speed,
    var_speed = var_speed,
)

plt.scatter(dts, dxs, s = 1)
plt.show()
# dxs
# LineString(segment).length
# avg_speed
# n = int(linestring.length // distance_delta) + 1
# subdivision = []
# for i in range(n):
#     if i == 0 or i == (n - 1):
#         subdivision.append(i)
#     else:
#         subdivision.append(i + np.random.normal(0, subdivision_variance))
# points = [
#     linestring.interpolate(
#         subdivision[i] / float(n - 1), normalized = True
#     ) for i in range(n)
# ]
# points_ = []
# for point in points:
#     rand = np.random.normal(0, datapoint_variance, size = 2)
#     points_.append(
#         [point.x + rand[0], point.y + rand[1]]
#     )
# return LineString(points_)

In [ ]:
# for coord in list(path.path.coords):
#     print(coord)
#     break

# path.path.length

avg_speed = 5.                      # Average human walking speed in km/h
avg_speed = 5. * (1000 / (60 * 60)) # Average human walking speed in m/s
var_speed = 0.5
samples = 2     # Count 
timeperiod = 10 # In seconds between datapoints
sampling_rate = samples / timeperiod  # Desired number of samples per second
print("Sampling rate: ", sampling_rate)
distance = avg_speed * timeperiod
distance

In [4]:

# map_ = setup_map(center)
# map_ = plot_point(start_dp, center, color = "red", map_ = map_)
# map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
# map_ = plot_polyline(
#     [start_dp] + [DataPoint(x, y) for x, y in polyline] + [end_dp],
#     center,
#     color = "blue",
#     map_ = map_,
# )
# map_

In [ ]:
# np.random.seed(20)
def sample_path_batch(M, N):
    dt = 1.0 / (N - 1)
    dt_sqrt = numpy.sqrt(dt)
    B = numpy.empty((M, N), dtype=numpy.float32)
    B[:, 0] = 0
    for n in range(N - 2):                                           
        t = n * dt
        xi = numpy.random.randn(M) * dt_sqrt
        B[:, n + 1] = B[:, n] * (1 - dt / (1 - t)) + xi
    B[:, -1] = 0
    return B

# nvals = sample_path_batch(1, 10 + 1)[0]
# ovals = [i + 10 for i in range(2)]
# uvals = [i * 10 for i in range(len(ovals))]

# # using the variable axs for multiple Axes
# fig, axs = plt.subplots(3, 1, sharex = True, sharey = True)

# axs[0].scatter(uvals, ovals)
# axs[1].scatter([i for i in range(len(nvals))], nvals)

# avals = []
# for i in range(2, len(ovals) + 1):
#     vvals = ovals[i - 2:i]
# #     min_lon = np.min(ovals[i - 2:i])
# #     max_lon = np.max(ovals[i - 2:i])
# #     min_lat = np.min(ovals[i - 2:i])
# #     max_lat = np.max(ovals[i - 2:i])
#     for j in range(len(nvals)):
#         j_val = nvals[j]
#         # min_val = np.min(nvals)
#         # max_val = np.max(nvals)
#         nval = j_val + (vvals[0] *(uvals[1] - j) + vvals[1] * (j - uvals[0])) / (uvals[1] - uvals[0])
#         # nval = min_lon + ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
#         avals.append(nval)

# axs[2].scatter([i for i in range(len(avals))], avals)

# # plt.scatter([i for i in range(len(values))], values)
# # plt.plot([i for i in range(len(values))], values)
# plt.show()


In [ ]:
print(transformed_values_lon)

In [ ]:
# lon_nvals = []
# lat_nvals = []
# dist = 10.
# for i in range(2, len(polyline) + 1):
#     min_lon = np.min(lon[i - 2:i])
#     max_lon = np.max(lon[i - 2:i])
#     min_lat = np.min(lat[i - 2:i])
#     max_lat = np.max(lat[i - 2:i])
#     # NOTE: Use one brownian bridge at the moment
#     values = sample_path_batch(1, 3)[0]
#     for j in range(len(values)):
#         j_val = values[j]
#         min_val = np.min(values)
#         max_val = np.max(values)
#         nval = min_lon + ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
#         if j != 0 and j != len(values):
#             lon_nvals.append(nval)
#     values = sample_path_batch(1, 3)[0]
#     for j in range(len(values)):
#         j_val = values[j]
#         min_val = np.min(values)
#         max_val = np.max(values)
#         nval = min_lat + ((j_val - min_val) * (max_lat - min_lat)) / (max_val - min_val)
#         if j != 0 and j != len(values):
#             lat_nvals.append(nval)
#     if i == 1:
#         break

# coords = zip(lat_nvals, lon_nvals)
# # haversine_distance()

# #     print(min_lon - min_lon / (max_lon - min_lon))
# #     print(max_lon - min_lon / (max_lon - min_lon))
# #     print(lon[i - 2:i])

In [ ]:
lon_nvals = []
lat_nvals = []
param = 100
# for i in range(2, len(polyline) + 1):
for i in range(2, len(list(npolyline.coords)) + 1):
    min_lon = np.min(lon[i - 2:i])
    max_lon = np.max(lon[i - 2:i])
    min_lat = np.min(lat[i - 2:i])
    max_lat = np.max(lat[i - 2:i])
    lon_vals = lon[i - 2:i]
    lat_vals = lat[i - 2:i]
    indices = [i for i in range(param + 1)]
    values = sample_path_batch(1, param + 1)[0]
    # NOTE: Use one brownian bridge at the moment
    for j in range(len(values)):
        j_val = values[j]
        min_val = np.min(values)
        max_val = np.max(values)
#         nval = (lon_vals[0] * (indices[-1] - j) + lon_vals[1] * (j - indices[0])) / (indices[-1] - indices[0])
#         rej_val = min_lon + ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
        rej_val = ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
        nval = rej_val + (lon_vals[0] * (indices[-1] - j) + lon_vals[1] * (j - indices[0])) / (indices[-1] - indices[0])
        if j != 0:# and j != len(values):
            lon_nvals.append(nval)
    values = sample_path_batch(1, param + 1)[0]
    for j in range(len(values)):
        j_val = values[j]
        min_val = np.min(values)
        max_val = np.max(values)
        rej_val = ((j_val - min_val) * (max_lat - min_lat)) / (max_val - min_val)
        nval = rej_val + (lat_vals[0] * (indices[-1] - j) + lat_vals[1] * (j - indices[0])) / (indices[-1] - indices[0])
        if j != 0: # and j != len(values):
            lat_nvals.append(nval)

new_latvals = []
new_lonvals = []
for i in range(len(lat_nvals)):
    x, y = utm.to_latlon(lat_nvals[i], lon_nvals[i], 32, "U")
    new_latvals.append(x)
    new_lonvals.append(y)

# coords = zip(lat_nvals, lon_nvals)
coords = zip(new_latvals, new_lonvals)

# # plt.plot(lon_nvals, lat_nvals)
# plt.plot(new_latvals, new_lonvals)
# plt.show()
# len(list(npolyline.coords))
# len(list(lat_nvals))
# for item in coords:
#     print(item)

In [ ]:
map_ = setup_map(center)
map_ = plot_point(start_dp, center, color = "red", map_ = map_)
map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
map_ = plot_polyline(
    [start_dp] + [DataPoint(x, y) for x, y in coords] + [end_dp],
    center,
    color = "blue",
    map_ = map_,
)
map_
# map_ = setup_map(center)
# map_ = plot_point(start_dp, center, color = "red", map_ = map_)
# map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
# for point in [DataPoint(x, y) for x, y in coords]:
#     map_ = plot_point(point, center, color = "green", map_ = map_)
# map_ = plot_polyline([DataPoint(x, y) for x, y in coords], center, color = "blue", map_ = map_)
# map_
# plt.scatter([i for i in range(len(lat_nvals))], lat_nvals, s= 5)

# plt.scatter(lon[:1], lat[:1], s = 25, color = "red")
# plt.scatter(lon_nvals, lat_nvals, s = 5)
# plt.scatter([i for i in range(len(lat[:10]))], lat[:10], s = 25, color = "red")
# plt.scatter([i for i in range(len(lon_nvals))], lon_nvals, s = 5)

# plt.scatter([i for i in range(len(lat))], lat, s= 2.5)
# plt.plot(lon_nvals, lat_nvals)
# plt.show()

In [ ]:
# map_ = setup_map(center)
# map_ = plot_point(start_dp, center, color = "red", map_ = map_)
# map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
# map_ = plot_polyline(
#     [start_dp] + [DataPoint(x, y) for x, y in polyline] + [end_dp],
#     center,
#     color = "blue",
#     map_ = map_,
# )
# map_

In [97]:
from typing import Tuple, List, Union, Any, Dict
import logging
import json
import requests
import numpy as np
import pandas as pd
from requests.models import encode_multipart_formdata
from shapely.geometry import LineString, Polygon, Point
from folium import plugins
import folium
import numpy
import matplotlib.pyplot as plt
import utm
import random 
from typing import Tuple
import time
from functools import wraps


# Type aliases
Coordinates = Union[Tuple[float, float], List[float]]


# Use the following address when sending HTTP requests to Valhalla
OPTIMIZED_ROUTE_URL = "http://localhost:8002/optimized_route" 
DEFAULT_HEADERS = {"Content-type": "application/json"}

# Use six degrees of precision when using Valhalla for routing
VALHALLA_PRECISION = 1.0 / 1e6
EARTH_RADIUS = 6371. * 1000. # In meters


class DataPoint:
    
    def __init__(self, latitude: float, longitude: float) -> None:
        self.latitude = latitude
        self.longitude = longitude


class WayPoint(DataPoint):
    
    def __init__(
        self,
        latitude: float,
        longitude: float,
        duration: float = 0.,
        std: float = 0.
        ):
        super().__init__(latitude = latitude, longitude = longitude)
        if not duration >= 0.:
            raise ValueError(
                "A waypoint can not have a negative duration. Change it to a " + \
                "floating point value >= 0")
        self.std = std
        self.duration = duration
    
    def is_stop(self) -> bool:
        return self.duration > 0.


class ValhallaInterface:

    def __init__(self) -> None:
        pass

    def decode_polyline(self, polyline_string: str) -> List[Coordinates]:
        index = 0; latitude = 0; longitude = 0
        coordinates = []
        changes = {"latitude": 0, "longitude": 0}
        # Coordinates have variable length when encoded, so just keep
        # track of whether we have hit the end of the string. In each
        # while loop iteration a single coordinate is decoded.
        while index < len(polyline_string):
            # Gather latitude/longitufe changes, store them in a dictionary to apply them later
            for unit in ["latitude", "longitude"]: 
                shift, result = 0, 0
                while True:
                    byte = ord(polyline_string[index]) - 63
                    index += 1
                    result |= (byte & 0x1f) << shift
                    shift += 5
                    if not byte >= 0x20:
                        break
                if (result & 1):
                    changes[unit] = ~(result >> 1)
                else:
                    changes[unit] = (result >> 1)
            latitude += changes["latitude"]
            longitude += changes["longitude"]
            coordinates.append(
                [VALHALLA_PRECISION * latitude, VALHALLA_PRECISION * longitude],
            )
        return coordinates

    def send_optimized_route_request(self, wp0, wp1) -> Union[None, Any]:    
        def build_optimized_route_request(wp0, wp1):
            return json.dumps({
                "locations":[
                    # Start location
                    {"lat": wp0.latitude, "lon": wp0.longitude},
                    # End location
                    {"lat": wp1.latitude, "lon": wp1.longitude},
                ],
                "costing": "pedestrian",
                "directions_options": {
                    "units":"kilometers"
                },
            })
        d = build_optimized_route_request(wp0, wp1)
        response = requests.post(
            OPTIMIZED_ROUTE_URL,
            data = d,
            headers = DEFAULT_HEADERS,
        )
        if response.status_code == 200:
            content = json.loads(response.content)
        else:
            content = None
        return content

def setup_map(center, zoom_start = 14, tiles: str = "cartodbdark_matter"):
    map_ = folium.Map(
        location = center,
        zoom_start = zoom_start,
#         tiles = tiles,
    )
    plugins.Fullscreen(
        position = "topleft"
    ).add_to(map_)
    plugins.Draw(
        filename="placeholder.geojson",
        export = True,
        position = "topleft"
    ).add_to(map_)
    return map_

def plot_point(datapoint, center, color, radius = 5.0, opacity = 1, map_ = None):
    folium.CircleMarker(
        [datapoint.latitude, datapoint.longitude],
        radius = radius,
        color = color,
        opacity = opacity,
        popup = f"...",
    ).add_to(map_)
    return map_
    
def plot_polyline(datapoints, center, color, weight: float = 2.0, opacity: float = 1, map_ = None):
        lst = []
        for datapoint in datapoints:
            lst.append([
                datapoint.latitude,
                datapoint.longitude,
            ])
        folium.PolyLine(
            lst,
            color = color,
            weight = weight,
            opacity = opacity,
            popup = f"...",
        ).add_to(map_)
        return map_

    
def test_valhalla(start_dp, end_dp):
    d = json.dumps({
        "locations":[
            {"lat": start_dp.latitude, "lon": start_dp.longitude},
            {"lat": end_dp.latitude, "lon": end_dp.longitude},
#             {"lat": 55.39594, "lon": 10.38831},
#             {"lat": 55.39500, "lon": 10.38800},
        ],
        "costing": "pedestrian",
        "directions_options": {
            "units":"kilometers"
        },
    })
    response = requests.post(
        OPTIMIZED_ROUTE_URL,
        data = d,
        headers = DEFAULT_HEADERS,
    )
    return response

def haversine_distance(
        lat_1: float, lon_1: float,
        lat_2: float, lon_2: float,
    ) -> float:
    """Calculate the 'Haversine' (great-circle) distance in meters between two locations
    / (latitude, longitude) points.

    Args:
        lat_1 (float): Latitude of location 1.
        lon_1 (float): Longitude of location 1.
        lat_2 (float): Latitude of location 2.
        lon_2 (float): Longitude of location 2.

    Returns:
        float: The distance between location 1 and 2 in meters. 
    """
    d_lat = np.radians(lat_1 - lat_2)
    d_lon = np.radians(lon_1 - lon_2)
    lat1 = np.radians(lat_1)
    lat2 = np.radians(lat_2)
    a = np.sin(d_lat / 2) * np.sin(d_lat / 2) + \
        np.sin(d_lon / 2) * np.sin(d_lon / 2) * np.cos(lat1) * np.cos(lat2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = EARTH_RADIUS * c
    return d

def rand_24_bit() -> int:
    """Returns a random 24-bit integer"""
    return random.randrange(0, 16**6)


def color_dec() -> int:
    """Alias of rand_24 bit()"""
    return rand_24_bit()


def color_hex(num: int = rand_24_bit()) -> str:
    """Returns a 24-bit int in hex"""
    return "%06x" % num


def color_rgb(num: int = rand_24_bit()) -> Tuple[int, int, int]:
    """Returns three 8-bit numbers, one for each channel in RGB"""
    hx = color_hex(num)
    barr = bytearray.fromhex(hx)
    return (barr[0], barr[1], barr[2])


def timing(f):
    """ Define a decorator function to time a function call.

    Note:
        The function f is called in an ordinary manner and the result is returned.
        The time of the function call is simply written to a log.

    Args:
        f (function): The function to be timed.

    Returns:
        wrapper (function): The result of the function f.
    """
    @wraps(f)
    def wrapper(*args, **kwargs):
        start_sysusr = time.process_time(); start_wall = time.time()
        result = f(*args, **kwargs)
        end_sysusr = time.process_time(); end_wall = time.time()
        print(
#             "INFO: Call to " + str(f) + ". Elapsed time (sysusr): " + \
#             str(end_sysusr - start_sysusr) + " Elapsed time (wall): " + \
            " Elapsed time (wall): " + str(end_wall - start_wall))
        return result
    return wrapper

class Movement:
    
    def __init__(
        self,
        waypoints: List[WayPoint],
        mean_time_delta: float,
        std_time_delta: float,
        mean_speed: float,
        std_speed: float,
        std_datapoint: float,
        ) -> None:
        self.waypoints = waypoints
        self.mean_time_delta = mean_time_delta
        self.std_time_delta = std_time_delta
        self.mean_speed = mean_speed
        self.std_speed = std_speed
        self.std_datapoint = std_datapoint
        self.vi = ValhallaInterface()
        self.trajectory, self.timestamps = self.create_path()
    
    def stop_point(
        self,
        x: float,
        y: float,
        duration: float,
        std_waypoint: float,
        ) -> Tuple[List[Coordinates], List[float]]:
        _dts = 0.; dts = []; counter = 0
        coordinates = []
        while True:
            dt = self.mean_time_delta + np.abs(
                np.random.normal(0, self.std_time_delta)
            )
            dt_next = _dts + dt 
            if dt_next > duration:        
                break
            else:
                _dts += dt; dts.append(dt)
                coordinates.append([
                    x + np.random.normal(0, std_waypoint),
                    y + np.random.normal(0, std_waypoint),
                ])
                counter += 1        
        return coordinates, dts
    
    def create_segment(
        self,
        wp0: WayPoint,
        wp1: WayPoint,
        ) -> Tuple[List[Coordinates], List[float]]:
        content = self.vi.send_optimized_route_request(
            wp0 = wp0,
            wp1 = wp1,
        ) # TODO: Error handling. If content is None.
        polyline = self.vi.decode_polyline(
            content["trip"]["legs"][0]["shape"],
        )
        coordinates = []
        for coord in polyline[1:-1]:    
            x, y, _, _ = utm.from_latlon(coord[0], coord[1])
            coordinates.append(
                [x, y],
            )
        linestring, dts = self.interpolate(
            linestring = LineString(coordinates),
        )
        return list(linestring.coords), dts

    def _path_start(
        self,
        wp0: WayPoint,
        ) -> Dict[str, Any]:
        start_x, start_y, _, _ = utm.from_latlon(
            wp0.latitude, wp0.longitude,
        )
        start_coordinates, start_dts = self.stop_point(
            start_x, start_y,
            wp0.duration, wp0.std,
        )
        return {
            "start_x": start_x,
            "start_y": start_y,
            "start_coordinates": start_coordinates,
            "start_dts": start_dts, 
        }

    def _path_next(
        self,
        wp0: WayPoint,
        wp1: WayPoint,
        ) -> Dict[str, Any]:
        middle_coordinates, middle_dts = self.create_segment(
            wp0 = wp0,
            wp1 = wp1,
        )
        end_x, end_y, _, _ = utm.from_latlon(
            wp1.latitude, wp1.longitude,
        )
        end_coordinates, end_dts = self.stop_point(
            end_x, end_y,
            wp1.duration, wp1.std,
        )
        return {
            "end_x": end_x,
            "end_y": end_y,
            "middle_coordinates": middle_coordinates,
            "middle_dts": middle_dts,
            "end_coordinates": end_coordinates,
            "end_dts": end_dts,
        }

    def flatten(self, segments, timestamps) -> Tuple[LineString, List[float]]:
        _path = []; _timestamps = []
        for arr in timestamps:
            _timestamps.extend(arr)
        for arr in segments:
            _path.extend(arr)
        return LineString(_path), _timestamps

    def create_path(self):
        segments = []; timestamps = []
        if len(self.waypoints) >= 2:
            wp0 = self.waypoints[0]; wp1 = self.waypoints[1]
            for i in range(1, len(self.waypoints)):
                dict_ = self._path_next(
                    wp0 = self.waypoints[i - 1],
                    wp1 = self.waypoints[i],
                )            
                middle_coordinates = dict_["middle_coordinates"]
                middle_dts = dict_["middle_dts"]
                end_coordinates = dict_["end_coordinates"]
                end_dts = dict_["end_dts"]
                end_x = dict_["end_x"]
                end_y = dict_["end_y"]
                end_anchor = [[end_x, end_y]]
                if i == 1:
                    dict_ = self._path_start(
                        wp0 = wp0,
                    )
                    start_coordinates = dict_["start_coordinates"]
                    start_dts = dict_["start_dts"]
                    start_x = dict_["start_x"]
                    start_y = dict_["start_y"]
                    start_anchor = [[start_x, start_y]]
                    segment = \
                        start_coordinates + start_anchor + \
                        middle_coordinates + \
                        end_anchor + end_coordinates
                    start_dts[0] = 0
                    middle_dts[0] = self.mean_time_delta + \
                        np.abs(np.random.normal(0, self.std_time_delta))
                    dt0 = self.mean_time_delta + \
                        np.abs(np.random.normal(0, self.std_time_delta))
                    dt1 = self.mean_time_delta + \
                        np.abs(np.random.normal(0, self.std_time_delta))
                    dts = start_dts + [dt0] + middle_dts + [dt1] + end_dts
                else:
                    segment = \
                        middle_coordinates + \
                        end_anchor + end_coordinates
                    middle_dts[0] = self.mean_time_delta + \
                        np.abs(np.random.normal(0, self.std_time_delta))
                    dt0 = self.mean_time_delta + \
                        np.abs(np.random.normal(0, self.std_time_delta))
                    dts = middle_dts + [dt0] + end_dts
                if len(self.waypoints) > (i + 1):
                    latitude, longitude = utm.to_latlon(
                        end_coordinates[-1][0], end_coordinates[-1][1],
                        32, "U",
                    )
                    wp0 = WayPoint(
                        latitude = latitude,
                        longitude = longitude,
                        std = self.waypoints[i].std,
                        duration = self.waypoints[i].duration,
                    )
                    # Use the next waypoint
                    wp1 = self.waypoints[i + 1]
                segments.append(segment); timestamps.append(dts)
        else:
            raise ValueError(
                "At least two waypoints need to be provided."
            )
        return self.flatten(segments, timestamps)

    @timing
    def interpolate(
        self,
        linestring: LineString,
        ) -> Tuple[LineString, List[float]]:
        dxs, dts = self.segment_subdivision(linestring = linestring)
        points = [
            linestring.interpolate(dxs[i], normalized = True) for i in range(len(dxs))
        ]
        rand_arr = np.random.normal(0, self.std_datapoint, size = (len(points), 2))
        for i in range(rand_arr.shape[0]):
            rand_arr[i, 0] += points[i].x
            rand_arr[i, 1] += points[i].y
        return LineString(rand_arr.tolist()), dts

    def segment_subdivision(
        self,
        linestring: LineString,
        ) -> Tuple[List[float], List[float]]:
        dxs = [0.]; dts = [0.]; counter = 0
        # linestring = LineString(segment); linestring_length = linestring.length
        linestring_length = linestring.length
        while True:
            dt = self.mean_time_delta + np.abs(np.random.normal(0., self.std_time_delta))
            dx = (self.mean_speed + np.random.normal(0., self.std_speed)) * dt
            dx_next = dxs[counter] + dx 
            if dx_next > linestring_length:        
                break
            else:
                dts.append(dt)   
                dxs.append(dx_next)
                counter += 1
        return [v / linestring_length for v in dxs], dts
    
    def to_latlon(self) -> List[Coordinates]:
        coordinates = []
        if self.trajectory is not None:
            for coord in self.trajectory.coords:
                lat, lon = utm.to_latlon(coord[0], coord[1], 32, "U")
                coordinates.append([lat, lon])
        return coordinates

In [98]:
start_dp = DataPoint(
    latitude = 55.363451206145925,
    longitude = 10.412869042981713,
)
middle_dp = DataPoint(
    latitude = 55.373384555955425,
    longitude = 10.40937928427195,
)

end_dp = DataPoint(
    latitude = 55.37427658758254,
    longitude = 10.403394466623759,
)

waypoints = [
    WayPoint(
        latitude = 55.363451206145925,
        longitude = 10.412869042981713,
        duration = 120, # In seconds
        std = 14, # In meters
    ),
    WayPoint(
        latitude = 55.373384555955425,
        longitude = 10.40937928427195,
        duration = 120, # Seconds
        std = 14, # In meters
    ),
    WayPoint(
        latitude = 55.37427658758254,
        longitude = 10.403394466623759,
        duration = 120, # Seconds
        std = 14, # In meters
    )
]

avg_speed = 5.                      # Average human walking speed in km/h
avg_speed = 5. * (1000 / (60 * 60)) # Average human walking speed in m/s
var_speed = 0.5
var_speed = 0.5 * (1000 / (60 * 60)) # Walking speed in m/s

time_delta = 10
time_delta_variance = 25

map_ = setup_map(center)
map_ = plot_point(start_dp, center, color = "red", map_ = map_)
map_ = plot_point(middle_dp, center, color = "green", map_ = map_)
map_ = plot_point(end_dp, center, color = "blue", map_ = map_)

for _ in range(1):
    random.seed(time.time())
    color = "#" + str(color_hex(num = rand_24_bit()))
    path = Movement(
        waypoints = waypoints,
        mean_time_delta = time_delta,
        std_time_delta = time_delta_variance,
        mean_speed = avg_speed,
        std_speed = var_speed,
        std_datapoint = 20,
    )
    map_ = plot_polyline(
        [DataPoint(x, y) for x, y in list(path.to_latlon())],
        center,
        color = color,
        map_ = map_,
    )
map_

 Elapsed time (wall): 0.0039958953857421875
 Elapsed time (wall): 0.00095367431640625
